# Startup Success Prediction
Startups are young companies founded to develop a unique product or service with the intention to disrupt industries and change the world while doing it at scale. Startups aim to grow very fast starting with a product called minimum viable product (MVP) that will serve as a test to see whether the product is something the customer wants to use. From there, it will go through iterative growth an innovation looking to rapidly expand its customer base to establish itself in a larger market. The ultimate goal, implicitly or explicitly, is going public through a process known as Initial Public Offering (IPO). Going public offers a chance to investors to cash out, this is known as an exit.
<br>
Before going public, startups go through several rounds of funding where venture capital firms invest tens or hundreds of millions into these companies hoping to receive the more than 200,000% return Peter Thiel saw when he invested in Facebook 8 years before IPO. However, about 90% of startups fail. Therefore, investors have a very high chance of not getting any return on their investment. This model aims to help rich people get richer. Estimates from this model can then be used to determine whether it is worth investing in a startup.
<br>

### Objective
The objective is to predict whether a startup turns into a success or a failure. The success of a company is defined as the event that gives the company's founders and investors a large sum of money through the process of M&A (Merger and Acquisition) or an IPO. A company would be considered as failed if it had to be shut down.
<br>

### Dataset Description
The data contains industry trends, investment insights (total funding, number of investors, etc.), individual company information (location, industry, etc.), and whether or not the startup has been acquired.
<br>
The data set has 49 features. Most features in the dataset are self-explanatory. Here we list features that might need further description:
**age_first_funding_year**: Is the age of the company in years when it got its first funding.
**age_last_funding_year**: Is the age of the company in years when it got its last funding.
**has_VC**: Financed by venture capital firms.
**has_angel**: Financed by angel. An angel is a high-net-worth individual who invest in early stage companies.



# Data Exploration

In [335]:
import idlelib.tooltip

import pandas as pd
%matplotlib inline

df = pd.read_csv('data_startup.csv')
df.head()

,Unnamed: 0,state_code,latitude,longitude,zip_code,id,city,Unnamed: 6,name,labels,...,object_id,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,is_top500,status
0,1005,CA,42.358880,-71.056820,92101,c:6669,San Diego,NaN,Bandsintown,1,...,c:6669,0,1,0,0,0,0,1.0000,0,acquired
1,204,CA,37.238916,-121.973718,95032,c:16283,Los Gatos,NaN,TriCipher,1,...,c:16283,1,0,0,1,1,1,4.7500,1,acquired
2,1001,CA,32.901049,-117.192656,92121,c:65620,San Diego,San Diego CA 92121,Plixi,1,...,c:65620,0,0,1,0,0,0,4.0000,1,acquired
3,738,CA,37.320309,-122.050040,95014,c:42668,Cupertino,Cupertino CA 95014,Solidcore Systems,1,...,c:42668,0,0,0,1,1,1,3.3333,1,acquired
4,1002,CA,37.779281,-122.419236,94105,c:65806,San Francisco,San Francisco CA 94105,Inhale Digital,0,...,c:65806,1,1,0,0,0,0,1.0000,1,closed


### Cleanup
The dataset seems to have some redundant features:
Feature **Unnamed: 0** has unique values, ranging from 1 to 1153. Makes us wonder if this was a unique identifier, perhaps an index from a previous database. Safe to drop.

In [336]:
records_count = df.shape[0]
unnamed_0 = df.iloc[:,0].unique()
print("Records in dataset:", records_count)
print("Unique values in Unnamed: 0:", len(unnamed_0))
df.drop(['Unnamed: 0'], axis=1, inplace=True)

Records in dataset: 923
Unique values in Unnamed: 0: 923


Feature **Unnamed: 6** is missing some data but the feature seems to be just a merge of **city**, **state_code**, and **zip_code** features. Safe to drop.

In [337]:
df.loc[:, ['state_code', 'zip_code', 'city', 'Unnamed: 6']]

,state_code,zip_code,city,Unnamed: 6
0,CA,92101,San Diego,NaN
1,CA,95032,Los Gatos,NaN
2,CA,92121,San Diego,San Diego CA 92121
3,CA,95014,Cupertino,Cupertino CA 95014
4,CA,94105,San Francisco,San Francisco CA 94105
...,...,...,...,...
918,CA,94107,San Francisco,NaN
919,MA,1803,Burlington,Burlington MA 1803
920,CA,94089,Sunnyvale,NaN
921,CA,94404,San Francisco,NaN


In [338]:
df.drop(['Unnamed: 6'], axis=1, inplace=True)

Features **id** and **object_id** are identical. There is no need for either. Safe to drop both.

In [339]:
id_to_obj_id = df['id'] == df['object_id']
print("Records in dataset:", records_count)
print("Records where the id feature is equal to the object_id feature:", id_to_obj_id.sum())
df.drop(['id', 'object_id'], axis=1, inplace=True)

Records in dataset: 923
Records where the id feature is equal to the object_id feature: 923


The **labels** feature seems to be just a binary representation of the **status** feature. Safe to drop the labels feature.

In [340]:
df.loc[:, ['labels', 'status']][:20]

,labels,status
0,1,acquired
1,1,acquired
2,1,acquired
3,1,acquired
4,0,closed
5,0,closed
6,1,acquired
7,1,acquired
8,1,acquired
9,1,acquired


In [341]:
df.drop(['labels'], axis=1, inplace=True)

The state_code and state_code.1 features are identical. Safe to drop state_code.1

In [342]:
df.loc[:, ['state_code', 'state_code.1']][:20]

,state_code,state_code.1
0,CA,CA
1,CA,CA
2,CA,CA
3,CA,CA
4,CA,CA
5,CA,CA
6,CA,CA
7,CA,CA
8,MA,MA
9,CA,CA


In [343]:
state_comp = df['state_code'] == df['state_code.1']
print("Difference state_code vs state_code.1:", records_count - state_comp.sum(), "record.")
state_diff = df.loc[df['state_code'] != df['state_code.1']]
state_diff.loc[:, ['state_code', 'state_code.1']]

Difference state_code vs state_code.1: 1 record.


,state_code,state_code.1
515,CA,NaN


As shown above, further examination between the **state_code** and **state_code.1** features reveals a missing value in feature **state_code.1** in record 515. We still believe **state_code.1** should be droped.

In [344]:
df.drop(['state_code.1'], axis=1, inplace=True)

Records 832 and 124 are duplicates. Safe to drop one of them.

In [345]:
df.iloc[[124, 832]]

,state_code,latitude,longitude,zip_code,city,name,founded_at,closed_at,first_funding_at,last_funding_at,...,is_othercategory,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,is_top500,status
124,CA,37.54827,-121.988572,94538,Fremont,Redwood Systems,1/1/2008,NaN,10/3/2008,5/30/2012,...,1,1,0,1,1,1,0,2.25,1,acquired
832,CA,37.48151,-121.945328,94538,Fremont,Redwood Systems,1/1/2008,NaN,10/3/2008,5/30/2012,...,1,1,0,1,1,1,0,2.25,1,acquired


In [346]:
df.drop(832, axis=0, inplace=True)
# Note this creates a gap in the rows index. To reset the index use uncomment the following line:
# df = df.reset_index()

### Missing Values

In [347]:
print("Total missing values:", df.isnull().sum().sum())
missing_values = pd.DataFrame(df.isnull().sum(), columns=["Null Values"])
missing_values["% Missing Values"] = (df.isnull().sum() / len(df) * 100)
missing_values = missing_values[missing_values["% Missing Values"] > 0]
missing_values

Total missing values: 891


,Null Values,% Missing Values
closed_at,587,63.665944
age_first_milestone_year,152,16.485900
age_last_milestone_year,152,16.485900


##### Missing values in closed_at

In [348]:
closed_acquired = df.loc[(df['closed_at'].isnull()) & (df['status'] == 'acquired')]
closed_closed = df.loc[(df['closed_at'].isnull()) & (df['status'] == 'closed')]
print("Records where closed_at is missing and the startup status is acquired:", len(closed_acquired))
print("Records where closed_at is missing and the startup status is closed:", len(closed_closed))
df.loc[:, ['closed_at', 'status']][:20]

Records where closed_at is missing and the startup status is acquired: 587
Records where closed_at is missing and the startup status is closed: 0


,closed_at,status
0,NaN,acquired
1,NaN,acquired
2,NaN,acquired
3,NaN,acquired
4,10/1/2012,closed
5,2/15/2009,closed
6,NaN,acquired
7,NaN,acquired
8,NaN,acquired
9,NaN,acquired


Missing values in the **closed_at** feature seem to be related to a **status** of acquired. Makes sense, if the startup has been acquired or gone public, then it hasn't shut down. With over 63% missing values, it might be wiser to ignore this feature rather than try to impute an arbitrary value. Moreover, knowing when a startup shut down does not help us predict whether it will be successful or not. The startup has already failed.

In [353]:
df.drop(['closed_at'], axis=1, inplace=True)

##### Missing values in **age_first_milestone_year** and **age_last_milestone_year**

In [358]:
milestone_first_last = df.loc[(df['age_first_milestone_year'].isnull()) & (df['age_last_milestone_year'].isnull()) & (df['milestones'] == 0)]
print("Missing values in age_first_milestone_year and age_last_milestone_year with 0 milestones:", len(milestone_first_last))
milestone_first_last.loc[:, ['age_first_milestone_year', 'age_last_milestone_year', 'milestones']]

Missing values in age_first_milestone_year and age_last_milestone_year with 0 milestones: 152


,age_first_milestone_year,age_last_milestone_year,milestones
12,NaN,NaN,0
15,NaN,NaN,0
24,NaN,NaN,0
32,NaN,NaN,0
34,NaN,NaN,0
...,...,...,...
885,NaN,NaN,0
903,NaN,NaN,0
904,NaN,NaN,0
906,NaN,NaN,0


After an analysis of the data it was discovered that the same 152 records with missing values in **age_first_milestone_year** and **age_last_milestone_year** features have values of 0 in the **milestones** feature. We decided to zero out the missing values.

In [359]:
df['age_first_milestone_year'].fillna(value=0, inplace=True)
df['age_last_milestone_year'].fillna(value=0, inplace=True)
print("Total missing values:", df.isnull().sum().sum())

Total missing values: 0
